In [1]:
import pandas as pd
import re

In [9]:
data_name = '沈阳市工业和信息化局'  # '沈阳市科学技术局'  '沈阳市人民政府' '沈阳市工业和信息化局'

In [10]:
data = pd.read_csv(r'data/{}/{}.csv'.format(data_name,data_name), encoding='utf-8', header=0)
data

,title,content,Seg,Lac
0,标题：关于沈阳市2019年新增电力市场化交易用户准入名单的公示（第一批） 时间：[2019...,None,None,None
1,标题：沈工信发〔2019〕17号 时间：[2019-03-01] 来源：政策,沈工信发〔2019〕17号，此件不公开。,沈工信/ 发/ 〔/ 2019/ 〕/ 17号/ ，/ 此/ 件/ 不/ 公开/ 。,沈工信发(ORG)/ 〔(w)/ 2019(m)/ 〕(w)/ 17号(m)/ ，(w)/ ...
2,标题：党群携手 守护家园阵地 时间：[2020-03-23] 来源：政策,None,None,None
3,标题：我国首部绿色轮胎行业自律标准发布 时间：[2014-03-19] 来源：政策,2月24日，中国橡胶工业协会发布了备受国内外轮胎行业关注的《绿色轮胎技术规范》（以下简称《规...,2月24日/ ，/ 中国橡胶工业协会/ 发布/ 了/ 备受/ 国内外/ 轮胎/ 行业/ 关注...,2月24日(TIME)/ ，(w)/ 中国橡胶工业协会(ORG)/ 发布(v)/ 了(u)/...
4,标题：市经济和信息化委关于组织开展2014年节能服务公司推荐工作的通知 时间：[2014-...,"各有关单位：根据省经信委《关于组织开展2014年节能服务公司推荐工作的通知》要求,现将工业和...",各/ 有关/ 单位/ ：/ 根据/ 省经信委/ 《/ 关于/ 组织/ 开展/ 2014年/ ...,各(r)/ 有关(vn)/ 单位(n)/ ：(w)/ 根据(p)/ 省经信委(ORG)/ 《...
...,...,...,...,...
3475,标题：国家药管平台上线！想钻空子没门了 时间：[2016-12-15] 来源：政策,近日，国家药管平台网站已正式上线开通，这意味着我们成功迈进信息互联互通的药品严管时代。国家药...,近日/ ，/ 国家/ 药管/ 平台/ 网站/ 已/ 正式/ 上线/ 开通/ ，/ 这/ 意味...,近日(TIME)/ ，(w)/ 国家(n)/ 药管(vn)/ 平台(n)/ 网站(n)/ 已...
3476,标题：5G直播带货盛典今日启幕 时间：[2020-06-18] 来源：政策,6月18日，由市委宣传部、市委网信办指导，市商务局、市农业农村局和沈阳广电传媒集团共同主办的...,6月18日/ ，/ 由/ 市委宣传部/ 、/ 市委/ 网/ 信办/ 指导/ ，/ 市/ 商务...,6月18日(TIME)/ ，(w)/ 由(p)/ 市委(n)/ 宣传部(n)/ 、(w)/ ...
3477,标题：2017年贯彻落实《辽宁省优化营商环境条例》工作方案 时间：[2017-05-08]...,市经济和信息化委（2017年3月9日）为贯彻落实《辽宁省优化营商环境条例》、《辽宁省优化营商...,市/ 经济/ 和/ 信息化委/ （/ 2017年3月9日/ ）/ 为/ 贯彻/ 落实/ 《/...,市(n)/ 经济(n)/ 和(c)/ 信息化委(ORG)/ （(w)/ 2017年3月9日(...
3478,标题：关于申报2013年度电子信息产业发展基金项目的通知 时间：[2013-09-04] ...,各有关单位:根据工业和信息化部电子信息产业发展基金管理办公室《关于申报2013年度电子信息产...,各/ 有关/ 单位/ :/ 根据/ 工业/ 和/ 信息化部/ 电子信息产业/ 发展/ 基金/...,各(r)/ 有关(vn)/ 单位(n)/ :(w)/ 根据(p)/ 工业和信息化部电子信息产...


In [11]:
# segment用于可视化，publish_time为发布时间 
publish_time = re.compile(r'时间：\[?([\d-]{10})\]?')
def get_segments(content):
    content = re.sub("\([a-zA-Z]{1,4}\)", '', content)
    content_list = content.split('。')
    content_list = [ x + '。' for x in content_list]
    if content_list != ['None。'] :
        result = []
        temp = ''
        for x in content_list:
            if len(temp) < 200:
                temp += x
            else:
                result.append(temp)
                temp = x
        if temp and temp != '/ 。' and temp != '。':
            result.append(temp)
        return str(result)
    else:
        return str(['None'])
def get_publish_time(title):
    m = publish_time.findall(title)
    if len(m) > 0:
        return m[0]
    else:
        return 'None'
data['segments'] = data['Lac'].apply(get_segments)
data['publish_time'] = data['title'].apply(get_publish_time)
data

,title,content,Seg,Lac,segments,publish_time
0,标题：关于沈阳市2019年新增电力市场化交易用户准入名单的公示（第一批） 时间：[2019...,None,None,None,['None'],2019-03-05
1,标题：沈工信发〔2019〕17号 时间：[2019-03-01] 来源：政策,沈工信发〔2019〕17号，此件不公开。,沈工信/ 发/ 〔/ 2019/ 〕/ 17号/ ，/ 此/ 件/ 不/ 公开/ 。,沈工信发(ORG)/ 〔(w)/ 2019(m)/ 〕(w)/ 17号(m)/ ，(w)/ ...,['沈工信发/ 〔/ 2019/ 〕/ 17号/ ，/ 此/ 件/ 不/ 公开/ 。。'],2019-03-01
2,标题：党群携手 守护家园阵地 时间：[2020-03-23] 来源：政策,None,None,None,['None'],2020-03-23
3,标题：我国首部绿色轮胎行业自律标准发布 时间：[2014-03-19] 来源：政策,2月24日，中国橡胶工业协会发布了备受国内外轮胎行业关注的《绿色轮胎技术规范》（以下简称《规...,2月24日/ ，/ 中国橡胶工业协会/ 发布/ 了/ 备受/ 国内外/ 轮胎/ 行业/ 关注...,2月24日(TIME)/ ，(w)/ 中国橡胶工业协会(ORG)/ 发布(v)/ 了(u)/...,['2月24日/ ，/ 中国橡胶工业协会/ 发布/ 了/ 备受/ 国内外/ 轮胎/ 行业/ ...,2014-03-19
4,标题：市经济和信息化委关于组织开展2014年节能服务公司推荐工作的通知 时间：[2014-...,"各有关单位：根据省经信委《关于组织开展2014年节能服务公司推荐工作的通知》要求,现将工业和...",各/ 有关/ 单位/ ：/ 根据/ 省经信委/ 《/ 关于/ 组织/ 开展/ 2014年/ ...,各(r)/ 有关(vn)/ 单位(n)/ ：(w)/ 根据(p)/ 省经信委(ORG)/ 《...,['各/ 有关/ 单位/ ：/ 根据/ 省经信委/ 《/ 关于/ 组织/ 开展/ 2014年...,2014-07-08
...,...,...,...,...,...,...
3475,标题：国家药管平台上线！想钻空子没门了 时间：[2016-12-15] 来源：政策,近日，国家药管平台网站已正式上线开通，这意味着我们成功迈进信息互联互通的药品严管时代。国家药...,近日/ ，/ 国家/ 药管/ 平台/ 网站/ 已/ 正式/ 上线/ 开通/ ，/ 这/ 意味...,近日(TIME)/ ，(w)/ 国家(n)/ 药管(vn)/ 平台(n)/ 网站(n)/ 已...,['近日/ ，/ 国家/ 药管/ 平台/ 网站/ 已/ 正式/ 上线/ 开通/ ，/ 这/ ...,2016-12-15
3476,标题：5G直播带货盛典今日启幕 时间：[2020-06-18] 来源：政策,6月18日，由市委宣传部、市委网信办指导，市商务局、市农业农村局和沈阳广电传媒集团共同主办的...,6月18日/ ，/ 由/ 市委宣传部/ 、/ 市委/ 网/ 信办/ 指导/ ，/ 市/ 商务...,6月18日(TIME)/ ，(w)/ 由(p)/ 市委(n)/ 宣传部(n)/ 、(w)/ ...,['6月18日/ ，/ 由/ 市委/ 宣传部/ 、/ 市委/ 网/ 信办/ 指导/ ，/ 市...,2020-06-18
3477,标题：2017年贯彻落实《辽宁省优化营商环境条例》工作方案 时间：[2017-05-08]...,市经济和信息化委（2017年3月9日）为贯彻落实《辽宁省优化营商环境条例》、《辽宁省优化营商...,市/ 经济/ 和/ 信息化委/ （/ 2017年3月9日/ ）/ 为/ 贯彻/ 落实/ 《/...,市(n)/ 经济(n)/ 和(c)/ 信息化委(ORG)/ （(w)/ 2017年3月9日(...,['市/ 经济/ 和/ 信息化委/ （/ 2017年3月9日/ ）/ 为/ 贯彻/ 落实/ ...,2017-05-08
3478,标题：关于申报2013年度电子信息产业发展基金项目的通知 时间：[2013-09-04] ...,各有关单位:根据工业和信息化部电子信息产业发展基金管理办公室《关于申报2013年度电子信息产...,各/ 有关/ 单位/ :/ 根据/ 工业/ 和/ 信息化部/ 电子信息产业/ 发展/ 基金/...,各(r)/ 有关(vn)/ 单位(n)/ :(w)/ 根据(p)/ 工业和信息化部电子信息产...,['各/ 有关/ 单位/ :/ 根据/ 工业和信息化部电子信息产业发展基金管理办公室/ 《/...,2013-09-04


In [12]:
# 开源工具提取 PER、LOC、ORG、TIME
def get_PER(line):
    PER = [x.replace('(PER)', '') for x in line.split('/ ') if '(PER)' in x]
    if len(PER) > 0:
        return str(PER)
    else:
        return "['None']"
def get_LOC(line):
    LOC = [x.replace('(LOC)', '') for x in line.split('/ ') if '(LOC)' in x]
    if len(LOC) > 0:
        return str(LOC)
    else:
        return "['None']"
def get_ORG(line):
    ORG = [x.replace('(ORG)', '') for x in line.split('/ ') if '(ORG)' in x]
    if len(ORG) > 0:
        return str(ORG)
    else:
        return "['None']"
def get_TIME(line):
    TIME = [x.replace('(TIME)', '') for x in line.split('/ ') if '(TIME)' in x]
    if len(TIME) > 0:
        return str(TIME)
    else:
        return "['None']"
data['PER'] = data['Lac'].apply(get_PER)
data['LOC'] = data['Lac'].apply(get_LOC)
data['ORG'] = data['Lac'].apply(get_ORG)
data['TIME'] = data['Lac'].apply(get_TIME)
data

,title,content,Seg,Lac,segments,publish_time,PER,LOC,ORG,TIME
0,标题：关于沈阳市2019年新增电力市场化交易用户准入名单的公示（第一批） 时间：[2019...,None,None,None,['None'],2019-03-05,['None'],['None'],['None'],['None']
1,标题：沈工信发〔2019〕17号 时间：[2019-03-01] 来源：政策,沈工信发〔2019〕17号，此件不公开。,沈工信/ 发/ 〔/ 2019/ 〕/ 17号/ ，/ 此/ 件/ 不/ 公开/ 。,沈工信发(ORG)/ 〔(w)/ 2019(m)/ 〕(w)/ 17号(m)/ ，(w)/ ...,['沈工信发/ 〔/ 2019/ 〕/ 17号/ ，/ 此/ 件/ 不/ 公开/ 。。'],2019-03-01,['None'],['None'],['沈工信发'],['None']
2,标题：党群携手 守护家园阵地 时间：[2020-03-23] 来源：政策,None,None,None,['None'],2020-03-23,['None'],['None'],['None'],['None']
3,标题：我国首部绿色轮胎行业自律标准发布 时间：[2014-03-19] 来源：政策,2月24日，中国橡胶工业协会发布了备受国内外轮胎行业关注的《绿色轮胎技术规范》（以下简称《规...,2月24日/ ，/ 中国橡胶工业协会/ 发布/ 了/ 备受/ 国内外/ 轮胎/ 行业/ 关注...,2月24日(TIME)/ ，(w)/ 中国橡胶工业协会(ORG)/ 发布(v)/ 了(u)/...,['2月24日/ ，/ 中国橡胶工业协会/ 发布/ 了/ 备受/ 国内外/ 轮胎/ 行业/ ...,2014-03-19,['None'],['None'],['中国橡胶工业协会'],"['2月24日', '3月1日']"
4,标题：市经济和信息化委关于组织开展2014年节能服务公司推荐工作的通知 时间：[2014-...,"各有关单位：根据省经信委《关于组织开展2014年节能服务公司推荐工作的通知》要求,现将工业和...",各/ 有关/ 单位/ ：/ 根据/ 省经信委/ 《/ 关于/ 组织/ 开展/ 2014年/ ...,各(r)/ 有关(vn)/ 单位(n)/ ：(w)/ 根据(p)/ 省经信委(ORG)/ 《...,['各/ 有关/ 单位/ ：/ 根据/ 省经信委/ 《/ 关于/ 组织/ 开展/ 2014年...,2014-07-08,['赵子忠'],['None'],"['省经信委', 'jwhzch@163.com沈阳市经济和信息化委员会']","['2014年', '现', '7月9日', '2014年7月3日']"
...,...,...,...,...,...,...,...,...,...,...
3475,标题：国家药管平台上线！想钻空子没门了 时间：[2016-12-15] 来源：政策,近日，国家药管平台网站已正式上线开通，这意味着我们成功迈进信息互联互通的药品严管时代。国家药...,近日/ ，/ 国家/ 药管/ 平台/ 网站/ 已/ 正式/ 上线/ 开通/ ，/ 这/ 意味...,近日(TIME)/ ，(w)/ 国家(n)/ 药管(vn)/ 平台(n)/ 网站(n)/ 已...,['近日/ ，/ 国家/ 药管/ 平台/ 网站/ 已/ 正式/ 上线/ 开通/ ，/ 这/ ...,2016-12-15,['None'],['None'],['None'],"['近日', '未来']"
3476,标题：5G直播带货盛典今日启幕 时间：[2020-06-18] 来源：政策,6月18日，由市委宣传部、市委网信办指导，市商务局、市农业农村局和沈阳广电传媒集团共同主办的...,6月18日/ ，/ 由/ 市委宣传部/ 、/ 市委/ 网/ 信办/ 指导/ ，/ 市/ 商务...,6月18日(TIME)/ ，(w)/ 由(p)/ 市委(n)/ 宣传部(n)/ 、(w)/ ...,['6月18日/ ，/ 由/ 市委/ 宣传部/ 、/ 市委/ 网/ 信办/ 指导/ ，/ 市...,2020-06-18,['None'],"['沈阳', '沈阳', '沈阳市', '沈阳']",['沈阳广电传媒集团'],"['6月18日', '今年']"
3477,标题：2017年贯彻落实《辽宁省优化营商环境条例》工作方案 时间：[2017-05-08]...,市经济和信息化委（2017年3月9日）为贯彻落实《辽宁省优化营商环境条例》、《辽宁省优化营商...,市/ 经济/ 和/ 信息化委/ （/ 2017年3月9日/ ）/ 为/ 贯彻/ 落实/ 《/...,市(n)/ 经济(n)/ 和(c)/ 信息化委(ORG)/ （(w)/ 2017年3月9日(...,['市/ 经济/ 和/ 信息化委/ （/ 2017年3月9日/ ）/ 为/ 贯彻/ 落实/ ...,2017-05-08,['None'],"['辽宁省', '辽宁省', '沈阳', '沈阳', '沈阳市', '沈阳', '伊朗', ...","['信息化委', '市经信委', '华晨集团', '北方重工集团', '沈鼓集团', '东软...","['2017年3月9日', '2017年', '2017年', '2017年', '2017..."
3478,标题：关于申报2013年度电子信息产业发展基金项目的通知 时间：[2013-09-04] ...,各有关单位:根据工业和信息化部电子信息产业发展基金管理办公室《关于申报2013年度电子信息产...,各/ 有关/ 单位/ :/ 根据/ 工业/ 和/ 信息化部/ 电子信息产业/ 发展/ 基金/...,各(r)/ 有关(vn)/ 单位(n)/ :(w)/ 根据(p)/ 工业和信息化部电子信息产...,['各/ 有关/ 单位/ :/ 根据/ 工业和信息化部电子信息产业发展基金管理办公室/ 《/...,2013-09-04,['None'],['None'],"['工业和信息化部电子信息产业发展基金管理办公室', '市经信委', '省经信委', '信息...","['现', '2013年', '今年', '今后', '3月14', '日前', '3月20..."


In [13]:
# 规则提取 通知、解读政策、答复、批复
TongZhi = re.compile(r'关于(.+的?通知)')
JieDu = re.compile(r'(《.+》)')
DaFu = re.compile(r'关于(.+)的答复')
PiFu = re.compile(r'关于(.+)的批复')
contact_person  = re.compile(r'联\s?系\s?人：([^联系咨询电话]+)[联系咨询电话]{2,4}')
contact_number  = re.compile(r'[联系咨询]{0,4}电话\s?：([、 (—)\d-]+)')
Policy = re.compile(r'(《[^》]+》)')
def get_TongZhi(title):
    m = TongZhi.findall(title)
    if len(m) > 0:
        while '关于' in m[0]:
            m = TongZhi.findall(m[0])
        return m[0]
    else:
        return 'None'
def get_JieDu(title):
    if '通知' in title:
        return 'None'
    m = JieDu.findall(title)
    if m and '解读' in title:
        return m[0]              
    else:
        return 'None'
def get_DaFu(title):
    m = DaFu.findall(title)
    if len(m) > 0:
        return m[0]
    else:
        return 'None'
def get_PiFu(title):
    m = PiFu.findall(title)
    if len(m) > 0:
        return m[0]
    else:
        return 'None'
def get_contact_person(content):
    m = contact_person.findall(content)
    m = [x.replace('，','') for x in m]
    if len(m) > 0:
        return m[0]
    else:
        return 'None'
def get_contact_number(content):
    m = contact_number.findall(content)
    if len(m) > 0:
        return m[0]
    else:
        return 'None'
def get_Policy(content):
    m = Policy.findall(content)
    if len(m) > 0:
        return str(m)
    else:
        return "['None']"
data['TongZhi'] = data['title'].apply(get_TongZhi)
data['JieDu'] = data['title'].apply(get_JieDu)
data['DaFu'] = data['title'].apply(get_DaFu)
data['PiFu'] = data['title'].apply(get_PiFu)
data['contact_person'] = data['content'].apply(get_contact_person)
data['contact_number'] = data['content'].apply(get_contact_number)
data['Policy'] = data['content'].apply(get_Policy)
data

,title,content,Seg,Lac,segments,publish_time,PER,LOC,ORG,TIME,TongZhi,JieDu,DaFu,PiFu,contact_person,contact_number,Policy
0,标题：关于沈阳市2019年新增电力市场化交易用户准入名单的公示（第一批） 时间：[2019...,None,None,None,['None'],2019-03-05,['None'],['None'],['None'],['None'],None,None,None,None,None,None,['None']
1,标题：沈工信发〔2019〕17号 时间：[2019-03-01] 来源：政策,沈工信发〔2019〕17号，此件不公开。,沈工信/ 发/ 〔/ 2019/ 〕/ 17号/ ，/ 此/ 件/ 不/ 公开/ 。,沈工信发(ORG)/ 〔(w)/ 2019(m)/ 〕(w)/ 17号(m)/ ，(w)/ ...,['沈工信发/ 〔/ 2019/ 〕/ 17号/ ，/ 此/ 件/ 不/ 公开/ 。。'],2019-03-01,['None'],['None'],['沈工信发'],['None'],None,None,None,None,None,None,['None']
2,标题：党群携手 守护家园阵地 时间：[2020-03-23] 来源：政策,None,None,None,['None'],2020-03-23,['None'],['None'],['None'],['None'],None,None,None,None,None,None,['None']
3,标题：我国首部绿色轮胎行业自律标准发布 时间：[2014-03-19] 来源：政策,2月24日，中国橡胶工业协会发布了备受国内外轮胎行业关注的《绿色轮胎技术规范》（以下简称《规...,2月24日/ ，/ 中国橡胶工业协会/ 发布/ 了/ 备受/ 国内外/ 轮胎/ 行业/ 关注...,2月24日(TIME)/ ，(w)/ 中国橡胶工业协会(ORG)/ 发布(v)/ 了(u)/...,['2月24日/ ，/ 中国橡胶工业协会/ 发布/ 了/ 备受/ 国内外/ 轮胎/ 行业/ ...,2014-03-19,['None'],['None'],['中国橡胶工业协会'],"['2月24日', '3月1日']",None,None,None,None,None,None,"['《绿色轮胎技术规范》', '《规范》', '《规范》', '《规范》']"
4,标题：市经济和信息化委关于组织开展2014年节能服务公司推荐工作的通知 时间：[2014-...,"各有关单位：根据省经信委《关于组织开展2014年节能服务公司推荐工作的通知》要求,现将工业和...",各/ 有关/ 单位/ ：/ 根据/ 省经信委/ 《/ 关于/ 组织/ 开展/ 2014年/ ...,各(r)/ 有关(vn)/ 单位(n)/ ：(w)/ 根据(p)/ 省经信委(ORG)/ 《...,['各/ 有关/ 单位/ ：/ 根据/ 省经信委/ 《/ 关于/ 组织/ 开展/ 2014年...,2014-07-08,['赵子忠'],['None'],"['省经信委', 'jwhzch@163.com沈阳市经济和信息化委员会']","['2014年', '现', '7月9日', '2014年7月3日']",组织开展2014年节能服务公司推荐工作的通知,None,None,None,赵子忠,024-23768563,['《关于组织开展2014年节能服务公司推荐工作的通知》']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3475,标题：国家药管平台上线！想钻空子没门了 时间：[2016-12-15] 来源：政策,近日，国家药管平台网站已正式上线开通，这意味着我们成功迈进信息互联互通的药品严管时代。国家药...,近日/ ，/ 国家/ 药管/ 平台/ 网站/ 已/ 正式/ 上线/ 开通/ ，/ 这/ 意味...,近日(TIME)/ ，(w)/ 国家(n)/ 药管(vn)/ 平台(n)/ 网站(n)/ 已...,['近日/ ，/ 国家/ 药管/ 平台/ 网站/ 已/ 正式/ 上线/ 开通/ ，/ 这/ ...,2016-12-15,['None'],['None'],['None'],"['近日', '未来']",None,None,None,None,None,None,['None']
3476,标题：5G直播带货盛典今日启幕 时间：[2020-06-18] 来源：政策,6月18日，由市委宣传部、市委网信办指导，市商务局、市农业农村局和沈阳广电传媒集团共同主办的...,6月18日/ ，/ 由/ 市委宣传部/ 、/ 市委/ 网/ 信办/ 指导/ ，/ 市/ 商务...,6月18日(TIME)/ ，(w)/ 由(p)/ 市委(n)/ 宣传部(n)/ 、(w)/ ...,['6月18日/ ，/ 由/ 市委/ 宣传部/ 、/ 市委/ 网/ 信办/ 指导/ ，/ 市...,2020-06-18,['None'],"['沈阳', '沈阳', '沈阳市', '沈阳']",['沈阳广电传媒集团'],"['6月18日', '今年']",None,None,None,None,None,None,['None']
3477,标题：2017年贯彻落实《辽宁省优化营商环境条例》工作方案 时间：[2017-05-08]...,市经济和信息化委（2017年3月9日）为贯彻落实《辽宁省优化营商环境条例》、《辽宁省优化营商...,市/ 经济/ 和/ 信息化委/ （/ 2017年3月9日/ ）/ 为/ 贯彻/ 落实/ 《/...,市(n)/ 经济(n)/ 和(c)/ 信息化委(ORG)/ （(w)/ 2017年3月9日(...,['市/ 经济/ 和/ 信息化委/ （/ 2017年3月9日/ ）/ 为/ 贯彻/ 落实/ ...,2017-05-08,['None'],"['辽宁省', '辽宁省', '沈阳', '沈阳', '沈阳市', '沈阳', '伊朗', ...","['信息化委', '市经信委', '华晨集团', '北方重工集团', '沈鼓集团', '东软...","['2017年3月9日', '2017年', '2017年', '2017年', '2017...",None,None,None,None,None,None,"['《辽宁省优化营商环境条例》', '《辽宁省优化营商环境条例实施意见》', '《2017年..."
3478,标题：关于申报2013年度电子信息产业发展基金项目的通知 时间：[2013-09-04] ...,各有关单位:根据工业和信息化部电子信息产业发展基金管理办公室《关于申报2013年度电子信息产...,各/ 有关/ 单位/ :/ 根据/ 工业/ 和/ 信息化部/ 电子信息产业/ 发展/ 基金/...,各(r)/ 有关(vn)/ 单位(n)/ :(w)/ 根据(p)/ 工业和信息化部电子信息产...,['各/ 有关/ 单位/ :/ 根据/ 工业和信息化部电子信息产业发展基金管理办公室/ 《/...,2013-09-04,['None'],['None'],"['工业和信息化部电子信息产业发展基金管理办公室', '市经信委', '省经信委', '信息...","['现', '2013年', '今年', '今后', '3月14', '日前', '3月20...",申报2013年度电子信息产业发展基金项目的通知,None,None,None,None,None,"['《关于申报2013年度电子信息产业发展基金项目的通知》', '《2013年电子信息产业发..."


In [14]:
data[data["contact_number"] != 'None']["contact_number"]

4       024-23768563
38          22517210
84          22899010
90          22899010
106     024-23768563
            ...     
3408        82248085
3409        22858168
3441        22720724
3449        22720724
3450    024-23768563
Name: contact_number, Length: 122, dtype: object

In [15]:
data[data["contact_person"] != 'None']["contact_person"]

4       赵子忠
38      黄文举
84       樊萍
90       樊萍
106     赵子忠
       ... 
3409     刘意
3441     樊萍
3449     佟盛
3450    赵子忠
3464     刘巍
Name: contact_person, Length: 99, dtype: object

In [16]:
data.to_csv(r'data/{}/NER_{}.csv'.format(data_name,data_name),index=False, encoding='utf-8')

In [17]:
df = data[['TongZhi','JieDu','DaFu','PiFu','contact_person','contact_number','Policy']]
df

,TongZhi,JieDu,DaFu,PiFu,contact_person,contact_number,Policy
0,None,None,None,None,None,None,['None']
1,None,None,None,None,None,None,['None']
2,None,None,None,None,None,None,['None']
3,None,None,None,None,None,None,"['《绿色轮胎技术规范》', '《规范》', '《规范》', '《规范》']"
4,组织开展2014年节能服务公司推荐工作的通知,None,None,None,赵子忠,024-23768563,['《关于组织开展2014年节能服务公司推荐工作的通知》']
...,...,...,...,...,...,...,...
3475,None,None,None,None,None,None,['None']
3476,None,None,None,None,None,None,['None']
3477,None,None,None,None,None,None,"['《辽宁省优化营商环境条例》', '《辽宁省优化营商环境条例实施意见》', '《2017年..."
3478,申报2013年度电子信息产业发展基金项目的通知,None,None,None,None,None,"['《关于申报2013年度电子信息产业发展基金项目的通知》', '《2013年电子信息产业发..."


In [18]:
df[data['JieDu'] != 'None']

,TongZhi,JieDu,DaFu,PiFu,contact_person,contact_number,Policy


In [19]:
noise = data [(data['TongZhi'] == 'None') &(data['DaFu'] == 'None') & (data['PiFu'] == 'None') 
            & (data['JieDu'] == 'None') ][['title','content']]
noise

,title,content
0,标题：关于沈阳市2019年新增电力市场化交易用户准入名单的公示（第一批） 时间：[2019...,None
1,标题：沈工信发〔2019〕17号 时间：[2019-03-01] 来源：政策,沈工信发〔2019〕17号，此件不公开。
2,标题：党群携手 守护家园阵地 时间：[2020-03-23] 来源：政策,None
3,标题：我国首部绿色轮胎行业自律标准发布 时间：[2014-03-19] 来源：政策,2月24日，中国橡胶工业协会发布了备受国内外轮胎行业关注的《绿色轮胎技术规范》（以下简称《规...
5,标题：辛国斌出席新能源汽车国家监测与动力蓄电池回收利用溯源综合管理平台启动会 时间：[20...,2018年7月31日，工业和信息化部在北京召开新能源汽车国家监测与动力蓄电池回收利用溯源综合...
...,...,...
3474,标题：李克强“报告”到这里，接连3次被掌声打断 时间：[2020-05-23] 来源：政策,这是一份迟来的《政府工作报告》。2020年5月22日上午，李克强总理在人民大会堂向十三届全国...
3475,标题：国家药管平台上线！想钻空子没门了 时间：[2016-12-15] 来源：政策,近日，国家药管平台网站已正式上线开通，这意味着我们成功迈进信息互联互通的药品严管时代。国家药...
3476,标题：5G直播带货盛典今日启幕 时间：[2020-06-18] 来源：政策,6月18日，由市委宣传部、市委网信办指导，市商务局、市农业农村局和沈阳广电传媒集团共同主办的...
3477,标题：2017年贯彻落实《辽宁省优化营商环境条例》工作方案 时间：[2017-05-08]...,市经济和信息化委（2017年3月9日）为贯彻落实《辽宁省优化营商环境条例》、《辽宁省优化营商...


In [20]:
noise.to_csv(r'data/{}/noise_{}.csv'.format(data_name,data_name),index=False, encoding='utf-8')

In [ ]:
# 规则测试

In [ ]:
TongZhi = re.compile(r'关于(.+的?通知)')
def get_TongZhi(title):
    m = TongZhi.findall(title)
    if len(m) > 0:
        while '关于' in m[0]:
            m = TongZhi.findall(m[0])
        return m[0]
    else:
        return 'None'
get_TongZhi('标题：图解:关于​沈阳市人民政府关于印发沈阳市促进全民健身和体育消费推动体育产业高质量发展若干措施的通知的解读  时间：2020-07-02  来源：政策')

In [ ]:
get_TongZhi('标题：关于《沈阳市人民政府办公厅转发市经济和信息化委关于沈阳市引导企业上云实施方案的通知》的解读  时间：2018-10-26  来源：政策')

In [ ]:
import re
JieDu = re.compile(r'(《.+》)')
def get_JieDu(title):
    if '通知' in title:
        return 'None'
    m = JieDu.findall(title)
    if m and '解读' in title:
        return m[0]              
    else:
        return 'None'
get_JieDu('标题：关于《沈阳市人民政府办公厅关于做好国家学前教育改革发展实验区工作的实施意见》的政策解读  时间：2016-11-16  来源：政策')

In [ ]:
get_JieDu('标题：【图解】关于沈阳市人民政府办公室关于印发沈阳市城区绿化工作方案的通知的解读  时间：2020-01-13  来源：政策')

In [ ]:
get_JieDu('标题：关于《沈阳市大力发展装配式建筑的工作方案》的政策解读  时间：2018-02-24  来源：政策')

In [ ]:
get_JieDu('标题：关于《沈阳市人民政府办公厅关于振兴沈阳地方戏曲的实施意见》的政策解读  时间：2016-10-12  来源：政策')

In [ ]:
DaFu = re.compile(r'关于(.+)的答复')
def get_DaFu(title):
    m = DaFu.findall(title)
    if len(m) > 0:
        return m[0]
    else:
        return 'None'
get_DaFu('标题：关于加强县域内获得国家高新技术企业资格后提高奖金的建议（第0911号）的答复  时间：2019-04-26  来源：政策')

In [ ]:
PiFu = re.compile(r'关于(.+)的批复')
def get_PiFu(title):
    m = PiFu.findall(title)
    if len(m) > 0:
        return m[0]
    else:
        return 'None'
get_PiFu('标题：沈阳市科学技术局关于成立沈阳市技术转移协会的批复(沈科发〔2019〕51号)  时间：2019-08-22  来源：政策')